<a href="https://colab.research.google.com/github/flohmannjr/tensorflow_curso/blob/main/TensorFlow_TL3_Scaling_Up.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TensorFlow Transfer Learning Scaling Up

In [1]:
import tensorflow as tf
import tensorflow_hub as hub

import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow.keras import Model, Sequential
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input
from tensorflow.keras.layers import RandomFlip, RandomHeight, RandomRotation, RandomWidth, RandomZoom
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import image_dataset_from_directory

from sklearn.metrics import classification_report

## Setup

In [2]:
plt.rcParams['figure.figsize'] = [8, 5]
plt.rcParams['figure.dpi'] = 100
plt.style.use('seaborn-darkgrid')

### Constantes

In [3]:
SEMENTE = 2008193

DIR_TREINO = '101_food_classes_10_percent/train'
DIR_TESTE  = '101_food_classes_10_percent/test'

TAMANHO_LOTE = 32
FORMATO_IMAGEM = (224, 224)
MODO_CLASSE = 'categorical'

FORMATO_ENTRADA = FORMATO_IMAGEM + (3,)
ATIVACAO = 'softmax'

PERDA = 'categorical_crossentropy'
METRICAS = ['accuracy']

# OTIMIZADOR = 'Adam'
APRENDIZADO = 0.001
APRENDIZADO_RESSINTONIZADO = 0.0001

ITERACOES = 5
ITERACOES_RESSINTONIZADAS = ITERACOES + 5

### Funções

In [4]:
!wget https://raw.githubusercontent.com/flohmannjr/tensorflow_curso/main/funcoes.py

--2023-01-25 00:50:57--  https://raw.githubusercontent.com/flohmannjr/tensorflow_curso/main/funcoes.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4971 (4.9K) [text/plain]
Saving to: ‘funcoes.py’

funcoes.py          100%[===================>]   4.85K  --.-KB/s    in 0s      

2023-01-25 00:50:57 (51.1 MB/s) - ‘funcoes.py’ saved [4971/4971]



In [5]:
from funcoes import grafico_historico_por_iteracao, grafico_historicos_complementares, unzip_data

## Dados

In [6]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip

unzip_data('101_food_classes_10_percent.zip')

--2023-01-25 00:50:57--  https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.163.128, 142.251.167.128, 172.253.115.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.163.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1625420029 (1.5G) [application/zip]
Saving to: ‘101_food_classes_10_percent.zip’

101_food_classes_10 100%[===================>]   1.51G   125MB/s    in 8.8s    

2023-01-25 00:51:06 (177 MB/s) - ‘101_food_classes_10_percent.zip’ saved [1625420029/1625420029]



### Importando os dados e tranformando-os em lotes

In [37]:
dados_treino = image_dataset_from_directory(directory=DIR_TREINO,
                                            batch_size=TAMANHO_LOTE,
                                            image_size=FORMATO_IMAGEM,
                                            label_mode=MODO_CLASSE,
                                            seed=SEMENTE)

dados_teste = image_dataset_from_directory(directory=DIR_TESTE,
                                           batch_size=TAMANHO_LOTE,
                                           image_size=FORMATO_IMAGEM,
                                           label_mode=MODO_CLASSE,
                                           shuffle=False, # Não embaralhar os dados de teste para permitir avaliarmos as previsões.
                                           seed=SEMENTE)

Found 7575 files belonging to 101 classes.
Found 25250 files belonging to 101 classes.


In [38]:
rotulos = dados_treino.class_names

## Pré-modelagem

### Expansão dos dados

In [ ]:
expansao_dados = Sequential(name='expansao_dados')

expansao_dados.add(RandomFlip('horizontal'))
expansao_dados.add(RandomHeight(0.2))
expansao_dados.add(RandomRotation(0.2))
expansao_dados.add(RandomWidth(0.2))
expansao_dados.add(RandomZoom(0.2))

### Modelo-base

In [ ]:
modelo_base = EfficientNetB0(include_top=False)
modelo_base.trainable = False

## Modelo inicial

In [ ]:
entradas = Input(shape=FORMATO_ENTRADA, name='camada_entrada')

# Há um bug na versão 2.8 do TensorFlow que faz necessário forçar o treinamento para que a expansão dos dados funcione.
expandidos = expansao_dados(entradas, training=True)

camadas = modelo_base(expandidos, training=False)
camadas = GlobalAveragePooling2D(name='agrupamento_media_global')(camadas)

saidas = Dense(len(rotulos), activation=ATIVACAO, name='camada_saida')(camadas)

modelo = Model(inputs=entradas, outputs=saidas, name='Modelo')

modelo.compile(loss=PERDA,
               optimizer=Adam(learning_rate=APRENDIZADO),
               metrics=METRICAS)

In [ ]:
historico_inicial = modelo.fit(dados_treino,
                               epochs=ITERACOES,
                               steps_per_epoch=len(dados_treino),
                               validation_data=dados_teste,
                               validation_steps=int(0.15 + len(dados_teste)), # Apenas 15% da base de teste
                               verbose=1)

In [ ]:
grafico_historico_por_iteracao(historico_inicial)

In [ ]:
validacao_inicial = modelo.evaluate(dados_teste)
validacao_inicial

## Ressintonizando modelo

In [ ]:
# "Descongela" as últimas 5 camadas do modelo_base (EfficientNetB0).
for layer in modelo_base.layers[-5:]:
    layer.trainable = True

modelo.compile(loss=PERDA,
               optimizer=Adam(learning_rate=APRENDIZADO_RESSINTONIZADO),
               metrics=METRICAS)

In [ ]:
historico_ressintonizado = modelo.fit(dados_treino,
                                     epochs=ITERACOES_RESSINTONIZADAS,
                                     steps_per_epoch=len(dados_treino),
                                     initial_epoch=len(historico_inicial.epoch),
                                     validation_data=dados_teste,
                                     validation_steps=int(0.15 * len(dados_teste)),
                                     verbose=1)

In [ ]:
grafico_historico_por_iteracao(historico_ressintonizado)

In [ ]:
grafico_historicos_complementares(historico_inicial, historico_ressintonizado)

In [ ]:
validacao_ressintonizada = modelo.evaluate(dados_teste)
validacao_ressintonizada

## Salvando modelo

In [ ]:
# Formato HDF5
modelo.save('scaling_up_HDF5.h5')

## Carregando modelo

In [ ]:
# modelo = tf.keras.models.load_model('scaling_up_HDF5.h5')

## Pretuned model

In [ ]:
# !wget https://storage.googleapis.com/ztm_tf_course/food_vision/06_101_food_class_10_percent_saved_big_dog_model.zip

# unzip_data('06_101_food_class_10_percent_saved_big_dog_model.zip')

# modelo = tf.keras.models.load_model('06_101_food_class_10_percent_saved_big_dog_model')

## Previsões

In [39]:
previsoes = modelo.predict(dados_teste, verbose=1)

790/790 [==============================] - 1544s 2s/step


In [40]:
# Retorna o índice da classe com maior probabilidade
classes_previstas = previsoes.argmax(axis=1)

`dados_teste` está no formato `BatchDataset`, que contém dois tensores: a imagem `(altura, largura, canais)` e a classe `(one-hot)`.

Para extrair os dados é necessário "deslotear" com `unbatch()` e iterar o resultado.

In [43]:
classes_verdadeiras = []

for imagem, classe in dados_teste.unbatch():
    classes_verdadeiras.append(classe.numpy().argmax())

In [49]:
print(classification_report(y_true=classes_verdadeiras,
                            y_pred=classes_previstas,
                            target_names=rotulos))

                         precision    recall  f1-score   support

              apple_pie       0.29      0.20      0.24       250
         baby_back_ribs       0.51      0.69      0.59       250
                baklava       0.56      0.65      0.60       250
         beef_carpaccio       0.74      0.53      0.62       250
           beef_tartare       0.73      0.43      0.54       250
             beet_salad       0.34      0.54      0.42       250
               beignets       0.67      0.79      0.72       250
               bibimbap       0.82      0.76      0.79       250
          bread_pudding       0.40      0.37      0.39       250
      breakfast_burrito       0.62      0.44      0.51       250
             bruschetta       0.62      0.42      0.50       250
           caesar_salad       0.84      0.49      0.62       250
                cannoli       0.52      0.74      0.61       250
          caprese_salad       0.56      0.60      0.58       250
            carrot_cake 